In [1]:
import argparse
import numpy as np
import sys
import time
import pprint
import itertools

import parameters
from OrbitalHessian import OrbitalHessian
import CppRowGen

In [2]:
def compare(params):
    if params.n_dimensions == 2:
        r = CppRowGen.CppRowGen2D(params, 1.0e-10)
    elif params.n_dimensions == 1:
        r = CppRowGen.CppRowGen1D(params, 1.0e-10)
    elif params.n_dimensions == 3:
        r = CppRowGen.CppRowGen3D(params, 1.0e-10)
        
    H = OrbitalHessian(params)
    np.set_printoptions(precision=4, linewidth=200)
    if params.instability_type == 'cRHF2cUHF':
        print('cRHF2cUHF')
        print('A')
        r.generate_TripletA(0, offset=0)
        cy = dict(zip(r.indices, r.values))
        py = dict(H.A.row_generator(0))
        d = {key: (cy[key], py[key]) for key in set(itertools.chain(cy.keys(), py.keys()))}
        pprint.pprint(d)
        print('B')
        r.generate_TripletB(0, offset=0)
        cy = dict(zip(r.indices, r.values))
        py = dict(H.B.row_generator(0))
        d = {key: (cy[key], py[key]) for key in set(itertools.chain(cy.keys(), py.keys()))}
        pprint.pprint(d)
        
    if params.instability_type == 'cRHF2cRHF':
        print('cRHF2cRHF')
        print('A')
        r.generate_SingletA(0, offset=0)
        cy = dict(zip(r.indices, r.values))
        py = dict(H.A.row_generator(0))
        d = {key: (cy[key], py[key]) for key in set(itertools.chain(cy.keys(), py.keys()))}
        pprint.pprint(d)
        print('B')
        r.generate_SingletB(0, offset=0)
        cy = dict(zip(r.indices, r.values))
        py = dict(H.B.row_generator(0))
        d = {key: (cy[key], py[key]) for key in set(itertools.chain(cy.keys(), py.keys()))}
        pprint.pprint(d)

params = parameters.Parameters(n_dimensions=3
                              ,n_k_points=7
                              ,instability_type='cRHF2cUHF'
                              ,safe_eri=False
                              )
        
#compare(params)

params = parameters.Parameters(n_dimensions=3
                              ,n_k_points=7
                              ,instability_type='cRHF2cRHF'
                              ,safe_eri=False
                              )

#compare(params)

In [3]:
from petsc4py import PETSc
from slepc4py import SLEPc
import cProfile

params = parameters.Parameters(n_dimensions=2
                              ,n_k_points=20
                              ,instability_type='cRHF2cRHF'
                              ,safe_eri=False
                              )

def np_fill(params):
    H = OrbitalHessian(params)
    return H.as_numpy()

def petsc_old(params):
    H = OrbitalHessian(params)
    return H.as_PETSc()

def petsc_fill(params):
    mat = PETSc.Mat()
    mat.create(PETSc.COMM_WORLD)
    mat.setSizes([2*params.excitations.n, 2*params.excitations.n])
    mat.setUp()
    r = CppRowGen.CppRowGen2D(params, 1.0e-10)
    for i in range(0, params.excitations.n):
        r.generate_TripletA(i)
        mat.setValues(i, r.indices, r.values)
        r.generate_TripletB(i, offset=params.excitations.n)
        mat.setValues(i, r.indices, r.values)
        '''
        py = {i : v for i, v in H.B.row_generator(i)}
        cy = {i : v for i, v in zip(r.indices, r.values)}
        keys = set(itertools.chain(py.keys(), cy.keys()))
        both = {k: (cy[k], py[k]) for k in keys}
        diff = {k: v for k, v in both.items() if abs(v[0] - v[1]) > 1e-6}
        print('row {} :'.format(i))
        pprint.pprint(diff)
        '''

    for i in range(params.excitations.n, 2 * params.excitations.n):
        r.generate_TripletB(i - params.excitations.n)
        mat.setValues(i, r.indices, r.values)
        r.generate_TripletA(i - params.excitations.n, offset=params.excitations.n)
        mat.setValues(i, r.indices, r.values)

    mat.assemble()
    return mat


np.set_printoptions(precision=4)

#petsc = petsc_fill(params).convert('dense')
#p = petsc.getDenseArray()
#n = np_fill(params)
#print(np.all(np.isclose(p, n)))

In [4]:
%timeit parameters.Parameters(rs=1.2, n_dimensions=2,n_k_points=12,instability_type='cRHF2cUHF',safe_eri=False)

10 loops, best of 3: 21.4 ms per loop


In [5]:
%timeit petsc_fill(params)

1 loop, best of 3: 1.35 s per loop


In [6]:
import slepc_wrapper
params = parameters.Parameters(rs=1.2, n_dimensions=2,n_k_points=12,instability_type='cRHF2cUHF',safe_eri=False)

#h = OrbitalHessian(params)
#print(h.as_numpy())

p = petsc_fill(params)
pnp = p.convert('dense')
pnp = pnp.getDenseArray()
#print(np.linalg.eigvals(pnp))
e = slepc_wrapper.SlepcEPSWrapper(p)
e.solve()
#print(params)
#print('\n', e)
print(params.excitations.n)

175


In [7]:
%timeit e.solve()

The slowest run took 5.27 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 12.1 µs per loop


In [8]:
print(e)
print(params)

Number of iterations of the method: 7
Solution method: jd
Number of requested eigenvalues: 1
Stopping condition: tol=1e-08, maxit=99
Number of converged eigenpairs 1
[(-0.09473050488792646, 2.308811045107682e-09)]

--------------------------------------------------------------------------------
-                                  Parameters                                  -
--------------------------------------------------------------------------------
rs : 1.2
n_k_points : 12
n_dimensions : 2
safe_eri : False
instability_type : cRHF2cUHF
k_fermi : 1.1785113019775793
k_max : 2.3570226157402714
k_grid : [-2.357  -1.9642 -1.5713 -1.1785 -0.7857 -0.3928  0.      0.3928  0.7857
  1.1785  1.5713  1.9642]
k_grid_spacing : 0.392837102623
n_electrons : 50
volume : 226.1946710584651
eri : <twoERI.TwoElectronIntegral object at 0x7f9e070ccf98>


In [9]:
print(params.excitations.n**2)

30625
